In [1]:
!pip install transformers accelerate bitsandbytes
!pip install torch -U
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import f1_score, mean_squared_error
from tqdm import tqdm

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Load the test dataset
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
test_csv_path = '/content/drive/My Drive/test_dataset.csv'
test_df = pd.read_csv(test_csv_path)

In [5]:
# Initialize the model and tokenizer
model_name = "meta-llama/Llama-2-7b-chat-hf"
access_token = ""
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True, use_auth_token=access_token)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=access_token)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [18]:
y_true = []
y_pred = []
responses = []

for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    story = row['modified']
    position = row['position']
    prompt = f"There is one major continuity error, a lapse in the self-consistency of the narrative, in the story provided. Count the number of '<nl>' in the story that occurs before the continuity error. Return that number only. Do not return any other information in your response. Story: {story}"

    # Generate model output
    model_inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    output = model.generate(**model_inputs)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    print(response)
    responses.append(response)

    # Extract predicted position from the response
    try:
        predicted_position = int(''.join(filter(str.isdigit, response)))  # Extracts digits and converts to integer
        y_pred.append(predicted_position)
    except ValueError:
        y_pred.append(-1)  # Handle non-numeric responses

    y_true.append(position)

100%|██████████| 100/100 [23:07<00:00, 13.87s/it]


In [20]:
df_response = pd.DataFrame(responses, columns=['response'])
df_preds = pd.DataFrame(y_pred, columns=['preds'])
df_true = pd.DataFrame(y_true, columns=['true'])

df_response.to_csv("test_responses.csv", index=False)
df_preds.to_csv("test_preds.csv", index=False)
df_true.to_csv("test_true.csv", index=False)

from google.colab import files
files.download("test_preds.csv")
files.download("test_true.csv")
files.download("test_responses.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Extract quantitiative data from LLM output

In [2]:
!pip install pandas openpyxl

In [7]:
import pandas as pd
import re

# Load the test dataset
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
df = pd.read_excel("/content/drive/MyDrive/test_continuity_error.xlsx", engine='openpyxl')

Mounted at /content/drive


In [15]:
# Define a function to extract the last line and the number from each response
def extract_position(response):
    try:
        # Split the response into sentences, assuming that a sentence ends with a period followed by a space or the end of the string
        sentences = re.split(r'\.\s+', response.strip())
        # Extract the last sentence
        last_sentence = sentences[-1]
        # Extract the number from the last sentence using regular expression
        # This regex looks for one or more digits (\d+)
        position = re.findall(r'\d+', last_sentence)
        if position:
            # Assuming we want the last number found, since it's typically the relevant one in the last sentence
            return int(position[-1])  # Convert the last found number to an integer
        else:
            return 0  # In case no number is found
    except Exception:
        return 0  # In case of any error
# Apply the function to each response and create a new 'position' column
df['position'] = df['response'].apply(extract_position)

df.to_excel('/content/updated_file.xlsx', index=False, engine='openpyxl')  # Saves the updated DataFrame

Run Evals

In [27]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
df = pd.read_csv("/content/drive/MyDrive/test_llama_continuity_error.csv")

Mounted at /content/drive


In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import roc_auc_score, confusion_matrix
import numpy as np

# Convert true and predicted positions to arrays for scoring
y_true = df["position"]
y_pred = df["llama"]

# Classification Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)

# Regression Metrics
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r_squared = r2_score(y_true, y_pred)

# Print all metrics
print(f'Classification Metrics:\nAccuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}')
print(f'Regression Metrics:\nMSE: {mse}\nRMSE: {rmse}\nMAE: {mae}\nR-squared: {r_squared}')

Classification Metrics:
Accuracy: 0.06
Precision: 0.43233333333333335
Recall: 0.06
F1 Score: 0.04340755735492577
Regression Metrics:
MSE: 69.89
RMSE: 8.360023923410745
MAE: 6.47
R-squared: -0.6854813232077441
